# Génération du dataset :
 - Générer tous les états possibles
 - Filtrer pour garder seulement les états valides (atteignables et non terminaux)


In [ ]:
import itertools
import numpy as np
import pandas as pd

# Constantes pour le jeu
X = 1       # Joueur X
O = -1      # Joueur O
EMPTY = 0   # Case vide

# Lignes gagnantes
WIN_COMBOS = [
    [0,1,2], [3,4,5], [6,7,8],  # lignes
    [0,3,6], [1,4,7], [2,5,8],  # colonnes
    [0,4,8], [2,4,6]            # diagonales
]

def check_winner(board):
    """
    Vérifie s'il y a un gagnant sur le plateau.
    Retourne X, O ou 0 si aucun.
    """
    for line in WIN_COMBOS:
        if board[line[0]] != EMPTY and board[line[0]] == board[line[1]] == board[line[2]]:
            return board[line[0]]
    return 0

def game_over(board):
    """
    Vérifie si la partie est terminée (victoire ou match nul)
    """
    return check_winner(board) != 0 or EMPTY not in board

def is_valid_state(board):
    """
    Vérifie si un état est atteignable (nombre de X et O valide et pas deux gagnants)
    """
    count_x = sum(1 for c in board if c == X)
    count_o = sum(1 for c in board if c == O)

    # Le joueur X commence => X doit avoir 0 ou 1 coup d’avance
    if count_x < count_o or count_x > count_o + 1:
        return False

    winner = check_winner(board)
    if winner == X and count_x != count_o + 1:
        return False
    if winner == O and count_x != count_o:
        return False
    if check_winner(board) == X and check_winner(board) == O:
        return False

    return True

def minimax(board, player, alpha, beta):
    """
    Minimax avec élagage alpha-bêta
    Retourne (score, coup)
    """
    winner = check_winner(board)
    if winner == X:
        return 1, None
    elif winner == O:
        return -1, None
    elif EMPTY not in board:
        return 0, None

    best_move = None
    if player == X:
        max_eval = -np.inf
        for i in range(9):
            if board[i] == EMPTY:
                board[i] = X
                eval, _ = minimax(board, O, alpha, beta)
                board[i] = EMPTY
                if eval > max_eval:
                    max_eval = eval
                    best_move = i
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
        return max_eval, best_move
    else:
        min_eval = np.inf
        for i in range(9):
            if board[i] == EMPTY:
                board[i] = O
                eval, _ = minimax(board, X, alpha, beta)
                board[i] = EMPTY
                if eval < min_eval:
                    min_eval = eval
                    best_move = i
                beta = min(beta, eval)
                if beta <= alpha:
                    break
        return min_eval, best_move

dataset = []

# Génère toutes les combinaisons possibles du plateau (3^9)
all_states = list(itertools.product([X, O, EMPTY], repeat=9))

for board in all_states:
    board = list(board)

    # Filtrer seulement les états atteignables et non terminés
    if not is_valid_state(board) or game_over(board):
      continue

    # Déterminer le joueur courant
    count_x = board.count(X)
    count_o = board.count(O)
    current_player = X if count_x == count_o else O

    # Calculer le meilleur coup avec Minimax
    _, best_move = minimax(board.copy(), current_player, -np.inf, np.inf)

    if best_move is not None:
      row = board + [current_player, best_move]
      dataset.append(row)

# Création du DataFrame
columns = [f'cell_{i}' for i in range(9)] + ['current_player', 'best_move']
df = pd.DataFrame(dataset, columns=columns)

# Sauvegarder le dataset
df.to_csv("tic_tac_toe_dataset_full.csv", index=False)
print(f"✅ Dataset généré avec {len(df)} exemples valides.")


✅ Dataset généré avec 4520 exemples valides.


In [ ]:
df.head()

,cell_0,cell_1,cell_2,cell_3,cell_4,cell_5,cell_6,cell_7,cell_8,current_player,best_move
0,1,1,-1,1,1,-1,-1,-1,0,1,8
1,1,1,-1,1,1,-1,-1,0,0,-1,8
2,1,1,-1,1,1,-1,0,-1,0,-1,8
3,1,1,-1,1,1,0,-1,-1,0,-1,8
4,1,1,-1,1,1,0,-1,0,-1,-1,5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4520 entries, 0 to 4519
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   cell_0          4520 non-null   int64
 1   cell_1          4520 non-null   int64
 2   cell_2          4520 non-null   int64
 3   cell_3          4520 non-null   int64
 4   cell_4          4520 non-null   int64
 5   cell_5          4520 non-null   int64
 6   cell_6          4520 non-null   int64
 7   cell_7          4520 non-null   int64
 8   cell_8          4520 non-null   int64
 9   current_player  4520 non-null   int64
 10  best_move       4520 non-null   int64
dtypes: int64(11)
memory usage: 388.6 KB


In [ ]:
print(df['current_player'].value_counts())

current_player
 1    2423
-1    2097
Name: count, dtype: int64



# Verification des doublons et des nulls

In [ ]:
# Vérifier les doublons
print(f"Nombre de lignes dupliquées : {df.duplicated().sum()}")

# Vérifier les valeurs nulles
print(f"\nNombre de valeurs nulles par colonne :\n{df.isnull().sum()}")

# Entraînement d’un modèle de classification multiple avec Random Forest


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Séparer les caractéristiques (X) et la cible (y)
# On utilise toutes les colonnes sauf 'best_move' comme caractéristiques
X_global = df.drop(columns=['best_move'])
y_global = df['best_move']

# Diviser les données en ensembles d'entraînement et de test
X_train_global, X_test_global, y_train_global, y_test_global = train_test_split(
    X_global, y_global, test_size=0.2, random_state=42, stratify=y_global
) # Ajout de stratify pour garantir une distribution similaire de la cible

print(f"Taille de l'ensemble d'entraînement (global): {len(X_train_global)}")
print(f"Taille de l'ensemble de test (global): {len(X_test_global)}")

# Initialiser et entraîner le modèle Random Forest
# Utilisation d'un nombre raisonnable d'estimateurs et d'un random_state pour la reproductibilité
model_rf_global = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1) # n_jobs=-1 pour utiliser tous les cœurs

print("\nEntraînement du modèle Random Forest sur l'ensemble global...")
model_rf_global.fit(X_train_global, y_train_global)
print("Entraînement terminé.")

# Faire des prédictions sur l'ensemble de test
y_pred_global = model_rf_global.predict(X_test_global)

# Évaluer le modèle
print("\nÉvaluation du modèle Random Forest (global):")

# Précision (Accuracy)
accuracy_global = accuracy_score(y_test_global, y_pred_global)
print(f"Précision (Accuracy): {accuracy_global:.4f}")

# Rapport de classification (Precision, Recall, F1-score)
print("\nRapport de classification:")
print(classification_report(y_test_global, y_pred_global, zero_division=0)) # zero_division=0 gère les classes sans prédictions

# Matrice de confusion
print("\nMatrice de confusion:")
conf_matrix_global = confusion_matrix(y_test_global, y_pred_global)
print(conf_matrix_global)


Taille de l'ensemble d'entraînement (global): 3616
Taille de l'ensemble de test (global): 904

Entraînement du modèle Random Forest sur l'ensemble global...
Entraînement terminé.

Évaluation du modèle Random Forest (global):
Précision (Accuracy): 0.7511

Rapport de classification:
              precision    recall  f1-score   support

           0       0.71      0.93      0.80       216
           1       0.76      0.82      0.79       117
           2       0.81      0.78      0.80       135
           3       0.70      0.58      0.63        73
           4       0.76      0.80      0.78       143
           5       0.74      0.47      0.57        49
           6       0.80      0.70      0.74        73
           7       0.75      0.35      0.48        34
           8       0.78      0.55      0.64        64

    accuracy                           0.75       904
   macro avg       0.76      0.66      0.69       904
weighted avg       0.75      0.75      0.74       904


Matrice de c

In [50]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

models_params = {
    "Random Forest": {
        "model": RandomForestClassifier(random_state=42, n_jobs=-1),
        "params": {
            "n_estimators": [50, 100],
            "max_depth": [None, 10, 20],
            "min_samples_split": [2, 5]
        }
    },
    "Régression Logistique": {
        "model": LogisticRegression(max_iter=1000),
        "params": {
            "C": [0.01, 0.1, 1, 10],
            "solver": ["lbfgs", "liblinear"]
        }
    },
    "Arbre de Décision": {
        "model": DecisionTreeClassifier(random_state=42),
        "params": {
            "max_depth": [None, 5, 10, 20],
            "min_samples_split": [2, 5, 10]
        }
    },
    "K plus proches voisins (KNN)": {
        "model": KNeighborsClassifier(),
        "params": {
            "n_neighbors": [3, 5, 7],
            "weights": ["uniform", "distance"]
        }
    }
}

results = {}

print("🔍 Début de la recherche d'hyperparamètres et évaluation des modèles...\n")

for name, mp in models_params.items():
    print(f"=== {name} ===")

    grid = GridSearchCV(mp["model"], mp["params"], cv=3, n_jobs=-1, scoring='accuracy')
    grid.fit(X_train_global, y_train_global)

    best_model = grid.best_estimator_
    print(f"Meilleurs paramètres : {grid.best_params_}")

    y_pred = best_model.predict(X_test_global)

    accuracy = accuracy_score(y_test_global, y_pred)
    print(f"Précision (Accuracy) sur test : {accuracy:.4f}")
    print("Rapport de classification :")
    print(classification_report(y_test_global, y_pred, zero_division=0))

    results[name] = {
        "best_params": grid.best_params_,
        "accuracy": accuracy,
        "classification_report": classification_report(y_test_global, y_pred, zero_division=0, output_dict=True)
    }
    print("-" * 50)

print("✅ Expérimentation terminée.")


🔍 Début de la recherche d'hyperparamètres et évaluation des modèles...

=== Random Forest ===
Meilleurs paramètres : {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Précision (Accuracy) sur test : 0.7511
Rapport de classification :
              precision    recall  f1-score   support

           0       0.71      0.93      0.80       216
           1       0.76      0.82      0.79       117
           2       0.81      0.78      0.80       135
           3       0.70      0.58      0.63        73
           4       0.76      0.80      0.78       143
           5       0.74      0.47      0.57        49
           6       0.80      0.70      0.74        73
           7       0.75      0.35      0.48        34
           8       0.78      0.55      0.64        64

    accuracy                           0.75       904
   macro avg       0.76      0.66      0.69       904
weighted avg       0.75      0.75      0.74       904

----------------------------------------------

# Le modèle final en utilisant le DataFrame entier sans le diviser, puisque c’est la version finale


In [ ]:
import numpy as np
# Séparer les caractéristiques (X) et la cible (y) en utilisant le DataFrame entier `df`
# On utilise toutes les colonnes sauf 'best_move' comme caractéristiques
X_full = df.drop(columns=['best_move'])
y_full = df['best_move']

print(f"Taille de l'ensemble de données complet pour l'entraînement final: {len(X_full)}")

# Initialiser et entraîner le modèle Random Forest sur l'ensemble de données complet
model_rf_final = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1) # n_jobs=-1 pour utiliser tous les cœurs

print("\nEntraînement du modèle Random Forest sur l'ensemble de données complet pour la version finale...")
model_rf_final.fit(X_full, y_full)
print("Entraînement final terminé.")

Taille de l'ensemble de données complet pour l'entraînement final: 4520

Entraînement du modèle Random Forest sur l'ensemble de données complet pour la version finale...
Entraînement final terminé.


In [ ]:
# un petit jeu tic tac toe console utilisant le model comme ia où le model joue le O et moi le X

import numpy as np
def print_board(board):
    """Affiche le plateau de jeu."""
    symbols = {X: 'X', O: 'O', EMPTY: ' '}
    print("\n")
    print(f" {symbols[board[0]]} | {symbols[board[1]]} | {symbols[board[2]]}")
    print("-----------")
    print(f" {symbols[board[3]]} | {symbols[board[4]]} | {symbols[board[5]]}")
    print("-----------")
    print(f" {symbols[board[6]]} | {symbols[board[7]]} | {symbols[board[8]]}")
    print("\n")

def get_human_move(board):
    """Demande au joueur humain de faire un coup valide."""
    while True:
        try:
            move = int(input("Entrez votre coup (0-8): "))
            if 0 <= move <= 8 and board[move] == EMPTY:
                return move
            else:
                print("Coup invalide. Veuillez choisir une case vide entre 0 et 8.")
        except ValueError:
            print("Entrée invalide. Veuillez entrer un nombre.")

def get_ai_move(board, model):
    """Fait jouer l'IA (modèle) et retourne son coup."""
    # L'IA joue O (-1)
    current_player = O
    # Le modèle a besoin des 9 cellules + le joueur actuel comme entrée
    # Convertir le plateau en numpy array pour l'entrée du modèle
    board_for_model = np.array(board + [current_player]).reshape(1, -1)

    # Faire la prédiction du meilleur coup
    predicted_move = model.predict(board_for_model)
    return predicted_move[0] # Le modèle retourne un array, on prend le premier élément

def play_game(model):
    """Déroule une partie de Tic Tac Toe."""
    board = [EMPTY] * 9
    current_player = X # Humain commence (X)

    print("Bienvenue au Tic Tac Toe!")
    print("Vous jouez 'X'. L'IA joue 'O'.")
    print("Les positions sur le plateau sont numérotées de 0 à 8:")
    print(" 0 | 1 | 2")
    print("-----------")
    print(" 3 | 4 | 5")
    print("-----------")
    print(" 6 | 7 | 8")


    while not game_over(board):
        print_board(board)

        if current_player == X:
            print("Votre tour (X)")
            move = get_human_move(board)
            board[move] = X
            current_player = O # Passer au joueur suivant
        else: # IA joue (O)
            print("Tour de l'IA (O)...")
            # Utiliser le modèle pour obtenir le coup de l'IA
            move = get_ai_move(board, model)
            print(f"L'IA joue la position {move}")
            board[move] = O
            current_player = X # Passer au joueur suivant


    print_board(board)
    winner = check_winner(board)
    if winner == X:
        print("Félicitations! Vous avez gagné!")
    elif winner == O:
        print("L'IA a gagné!")
    else:
        print("Match nul!")

# Lancer le jeu
play_game(model_rf_final)


Bienvenue au Tic Tac Toe!
Vous jouez 'X'. L'IA joue 'O'.
Les positions sur le plateau sont numérotées de 0 à 8:
 0 | 1 | 2
-----------
 3 | 4 | 5
-----------
 6 | 7 | 8


   |   |  
-----------
   |   |  
-----------
   |   |  


Votre tour (X)
Entrez votre coup (0-8): 4


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(




   |   |  
-----------
   | X |  
-----------
   |   |  


Tour de l'IA (O)...
L'IA joue la position 0


 O |   |  
-----------
   | X |  
-----------
   |   |  


Votre tour (X)
Entrez votre coup (0-8): 1


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(




 O | X |  
-----------
   | X |  
-----------
   |   |  


Tour de l'IA (O)...
L'IA joue la position 7


 O | X |  
-----------
   | X |  
-----------
   | O |  


Votre tour (X)
Entrez votre coup (0-8): 6


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(




 O | X |  
-----------
   | X |  
-----------
 X | O |  


Tour de l'IA (O)...
L'IA joue la position 2


 O | X | O
-----------
   | X |  
-----------
 X | O |  


Votre tour (X)
Entrez votre coup (0-8): 3


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(




 O | X | O
-----------
 X | X |  
-----------
 X | O |  


Tour de l'IA (O)...
L'IA joue la position 5


 O | X | O
-----------
 X | X | O
-----------
 X | O |  


Votre tour (X)
Entrez votre coup (0-8): 8


 O | X | O
-----------
 X | X | O
-----------
 X | O | X


Match nul!


In [ ]:
# Un petit jeu tic tac toe console utilisant le model comme ia où le model joue le X et moi le O (le model joue en premier)

import numpy as np
def play_game_console(model):
    """Déroule une partie de Tic Tac Toe en console."""
    board = [EMPTY] * 9
    #IA joue X (1), Humain joue O (-1)
    current_player = X # L'IA commence (X)

    print("Bienvenue au Tic Tac Toe!")
    print("L'IA joue 'X'. Vous jouez 'O'.")
    print("Les positions sur le plateau sont numérotées de 0 à 8:")
    print(" 0 | 1 | 2")
    print("-----------")
    print(" 3 | 4 | 5")
    print("-----------")
    print(" 6 | 7 | 8")


    while not game_over(board):
        print_board(board)

        if current_player == X: # IA joue (X)
            print("Tour de l'IA (X)...")
            # Le modèle est entraîné pour prédire le coup du *current_player*
            # Il faut donc lui passer le plateau ET le joueur actuel (X)
            board_for_model = np.array(board + [current_player]).reshape(1, -1)
            predicted_move = model.predict(board_for_model)
            move = predicted_move[0]

            if board[move] != EMPTY:
                 # Cas d'erreur: Le modèle a prédit un coup invalide.
                 # Cela ne devrait pas arriver avec un bon modèle entraîné sur Minimax,
                 # mais c'est une sécurité.
                 print(f"Erreur: L'IA a prédit un coup invalide sur la position {move}. Choix aléatoire d'une case vide.")
                 available_moves = [i for i, cell in enumerate(board) if cell == EMPTY]
                 if available_moves:
                     move = random.choice(available_moves)
                 else:
                     # La partie est déjà terminée, mais la boucle continue. On devrait sortir.
                     break # Sortir de la boucle si aucune case libre

            print(f"L'IA joue la position {move}")
            board[move] = X
            current_player = O # Passer au joueur suivant

        else: # Humain joue (O)
            print("Votre tour (O)")
            move = get_human_move(board)
            board[move] = O
            current_player = X # Passer au joueur suivant


    print_board(board)
    winner = check_winner(board)
    if winner == X:
        print("L'IA (X) a gagné!")
    elif winner == O:
        print("Félicitations! Vous (O) avez gagné!")
    else:
        print("Match nul!")

# Lancer le jeu avec l'IA jouant X et moi jouant O
play_game_console(model_rf_final)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Bienvenue au Tic Tac Toe!
L'IA joue 'X'. Vous jouez 'O'.
Les positions sur le plateau sont numérotées de 0 à 8:
 0 | 1 | 2
-----------
 3 | 4 | 5
-----------
 6 | 7 | 8


   |   |  
-----------
   |   |  
-----------
   |   |  


Tour de l'IA (X)...
L'IA joue la position 0


 X |   |  
-----------
   |   |  
-----------
   |   |  


Votre tour (O)
Entrez votre coup (0-8): 6


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(




 X |   |  
-----------
   |   |  
-----------
 O |   |  


Tour de l'IA (X)...
L'IA joue la position 1


 X | X |  
-----------
   |   |  
-----------
 O |   |  


Votre tour (O)
Entrez votre coup (0-8): 2


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(




 X | X | O
-----------
   |   |  
-----------
 O |   |  


Tour de l'IA (X)...
L'IA joue la position 4


 X | X | O
-----------
   | X |  
-----------
 O |   |  


Votre tour (O)
Entrez votre coup (0-8): 10
Coup invalide. Veuillez choisir une case vide entre 0 et 8.
Entrez votre coup (0-8): 8


 X | X | O
-----------
   | X |  
-----------
 O |   | O


Tour de l'IA (X)...
L'IA joue la position 7


 X | X | O
-----------
   | X |  
-----------
 O | X | O


L'IA (X) a gagné!


# Exportation du modèle final


In [ ]:
import numpy as np
import pickle

# Sauvegarder le modèle final
filename = 'final_tic_tac_toe_model.pkl'
pickle.dump(model_rf_final, open(filename, 'wb'))
print(f"Modèle final sauvegardé sous {filename}")



Modèle final sauvegardé sous final_tic_tac_toe_model.pkl

--- Exemple d'utilisation du modèle ---
Exemple 1: Plateau vide, joueur X commence.
Input pour le modèle: [[0 0 0 0 0 0 0 0 0 1]]
Coup prédit par le modèle: 0
Le modèle suggère que X devrait jouer à la position 0 dans un plateau vide.

Exemple 2: Plateau intermédiaire, joueur O doit jouer.
Input pour le modèle: [[ 1 -1  1 -1  1 -1  0  0  0 -1]]
Coup prédit par le modèle: 6
Le modèle suggère que O devrait jouer à la position 6 dans cet état du plateau.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
